# 自定义层

构造一个没有任何参数的自定义层

In [1]:
import mindspore
import mindspore.nn as nn
import mindspore.ops as ops

class CenteredLayer(nn.Cell):
    def __init__(self):
        super().__init__()

    def construct(self, X):
        return X - X.mean()

layer = CenteredLayer()
layer(mindspore.Tensor([1, 2, 3, 4, 5], dtype=mindspore.float32))

Tensor(shape=[5], dtype=Float32, value= [-2.00000000e+00, -1.00000000e+00,  0.00000000e+00,  1.00000000e+00,  2.00000000e+00])

将层作为组件合并到构建更复杂的模型中

In [2]:
import numpy as np
net = nn.SequentialCell([nn.Dense(8, 128), CenteredLayer()])

X = mindspore.Tensor(np.random.normal(0, 1, (4, 8)), mindspore.float32)
Y = net(X)
Y.mean()

Tensor(shape=[], dtype=Float32, value= -1.16415e-10)

带参数的层

In [3]:
from mindspore import Tensor, Parameter

class MyLinear(nn.Cell):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = Parameter(Tensor(np.random.normal(0, 1, (in_units, units)), mindspore.float32))
        self.bias = Parameter(Tensor(np.random.normal(0, 1, (units,)), mindspore.float32))

    def construct(self, X):
        linear = ops.matmul(X, self.weight) + self.bias
        return ops.ReLU()(linear)

linear = MyLinear(5, 3)
linear.weight

Parameter (name=weight, shape=(5, 3), dtype=Float32, requires_grad=True)

使用自定义层直接执行前向传播计算

In [4]:
X = Tensor(np.random.normal(0, 1, (2, 5)), mindspore.float32)

linear(X)

Tensor(shape=[2, 3], dtype=Float32, value=
[[ 1.32546091e+00,  3.91564280e-01,  0.00000000e+00],
 [ 0.00000000e+00,  8.59286726e-01,  2.44996011e-01]])

使用自定义层构建模型

In [5]:
net = nn.SequentialCell([MyLinear(64, 8), MyLinear(8, 1)])
X = Tensor(np.random.normal(0, 1, (2, 64)), mindspore.float32)
net(X)

Tensor(shape=[2, 1], dtype=Float32, value=
[[ 1.74028168e+01],
 [ 7.22708797e+00]])